# CSCI 3155 Recitation 9

November 2, 2018

## Type Inference

Given a program, the goal of type inference is to produce a mapping from program identifiers to types (or throw an error if no such mapping can be found). In a non-polymorphic language such as Lettuce, the types in the mapping should be concrete.

In lecture, we have seen a few informal examples of running type inference (Oct 25) along with a concrete implementation in Scala (Oct 30). The examples may have felt somewhat ad hoc in how they generated type variables; on the other hand, while the Scala implementation is fully general and systematic, it can be difficult to glean the essence of the algorithm from the code. Things are further complicated because the concrete implementation does not generate type variables and constraints in the same way that the examples do.

Let's consider the high-level algorithm of the implementation. This allows us to run through examples without having to generate type variables in an ad hoc manner, thereby developing a more precise intuition. We can think of the type inference algorithm as occurring in three phases. 

1. **Initialize type variables.** 
    - Generate a type variable for every identifier declaration (e.g. `f` and `x` in `let f = function (x) ... in ...`).
    - Collect all type variables in $V$.
    
2. **Generate constraints.** 
    - For each program expression in post-order traversal:
        - If the expression is a function application, create a new result type variable and add to $V$.
        - Tag the expression with its type (as an expression of type variables in $V$) along with any constraints imposed by that expression on the types of its subexpressions.
    - Collect all generated constraints in $C$.
3. **Solve constraints.**
    - Initialize empty substitution map $S$, which maps type variables to type expressions.
    - For each constraint in $C$:
        - Apply current substitutions to constraint.
        - If constraint is a tautology (e.g. `num == num`)
            - continue to next constraint                       
        - else if constraint is a contradiction (e.g. `num == bool`)
            - throw error
        - else if constraint can be unified (e.g. `t1 => t2 == t3 => t4`)
            - append unification constraints (e.g. `t1 == t3` and `t2 == t4`) to $C$
            - continue to next constraint
        - else
            - add constraint as new entry $e$ in $S$
            - update RHS of existing entries in $S$ with $e$
    - If $S$ does not consist solely of concrete types on the RHS, throw error.
    - Return $S$.

### Example 1

Let's run this high-level algorithm on the Lettuce program
```
(function (x) x+1)(5)
```

1. **Initialize type variables.**

    $V = \{t_x\}$
    
2. **Generate constraints.**

|Program expression (post-order)| Type | Constraints |
| --- | ----- | --- |
|`x`| $t_x$ | |
|`1`| num | |
|`x + 1`| num | $t_x = num, num => num$ |
|`function (x) x + 1`|  | | 
|`5`| num | | 
|`(function (x) x + 1)(5)`| $t_f$ | $c$ |

$\color{blue}{C = \{ t_x = num, num => num, t_x => num = num => t_f\}}$

3. **Solve constraints.**

    $S = \{\}$


| Constraint | Constraint w/ substitutions | Updated substitution map | Comment |
| --- | --- | --- | --- |
| $t_x = \texttt{num}$ | - | $\{t_x => num\}$ | |
| $\texttt{num} = \texttt{num}$ | |  | tautology |
| $t_x \rightarrow \texttt{num} = \texttt{num} \rightarrow t_{ap}$ | $num => num = num => t_f$ |  |  |
| $t_x \rightarrow \texttt{num} = \texttt{num} \rightarrow t_{ap}$ |  |  | tautology |
| $t_x \rightarrow \texttt{num} = \texttt{$t_f$} \rightarrow t_{ap}$ |  | $\{t_x => num, t_f => num\}$ |  |

### Example 2

```
let f = function (g) function (x) g(x) + 1 in
    let h = function (y) y * y in
        f(h)(5)
```

1. **Initialize type variables.**

    $V = \{\color{blue}{t_f, t_g, t_x, t_h, t_y}\}$
    
2. **Generate constraints.**

|Program expression (post-order)| Type | Constraints |
| --- | ----- | --- |
|`g`| $\color{blue}{t_{g}}$ | |
|`x`| $\color{blue}{t_x}$ | |
|`g(x)`| $\color{blue}{\text{$t_{g(x)}$ (new)}}$| $\color{blue}{t_g = t_x \rightarrow t_{g(x)}}$ |
|`1`| $\color{blue}{\texttt{num}}$ | | 
|`g(x) + 1`| $\color{blue}{\texttt{num}}$ | $\color{blue}{t_{g(x)} = \texttt{num},\ \ \texttt{num} = \texttt{num}}$ | 
|`function (x) g(x) + 1`| $\color{blue}{t_x \rightarrow \texttt{num}}$ | | 
|`function (g) function (x) g(x) + 1`| $\color{blue}{t_g \rightarrow t_x \rightarrow \texttt{num}}$ |  |
|`y`| $\color{blue}{t_{y}}$ | |
|`y * y`| $\color{blue}{\texttt{num}}$ | $\color{blue}{t_{y} = \texttt{num}}$|
| `function (y) y * y`| $\color{blue}{t_y \rightarrow \texttt{num}}$ | |
|`f`| $\color{blue}{t_{f}}$ | |
|`h`| $\color{blue}{t_h}$ | |
|`f(h)`| $\color{blue}{\text{$t_{f(h)}$ (new)}}$| $\color{blue}{t_f = t_h \rightarrow t_{f(h)}}$ |
|`5`| $\color{blue}{\texttt{num}}$ | | 
|`f(h)(5)`| $\color{blue}{\text{$t_{f(h)(5)}$ (new)}}$ | $\color{blue}{t_{f(h)} = \texttt{num} \rightarrow t_{f(h)(5)}}$ |
|`let h = function (y) y * y in f(h)(5)`| $\color{blue}{\text{$t_{f(h)(5)}$}}$ | $\color{blue}{t_{h} = \color{blue}{t_y \rightarrow \texttt{num}}}$
|`let f = ... in let h = ...` | $\color{blue}{\text{$t_{f(h)(5)}$}}$ | $\color{blue}{t_{f} = t_g \rightarrow t_x \rightarrow \texttt{num}}$

$\color{blue}{C = \{ t_g = t_x \rightarrow t_{g(x)},\ \ t_{g(x)} = \texttt{num},\ \ \texttt{num} = \texttt{num},\ \  t_{y} = \texttt{num},\ \ t_f = t_h \rightarrow t_{f(h)},\ \ t_{f(h)} = \texttt{num} \rightarrow t_{f(h)(5)},\ \ t_{h} = \color{blue}{t_y \rightarrow \texttt{num}},\ \ t_{f} = t_g \rightarrow t_x \rightarrow \texttt{num}\}}$

3. **Solve constraints.**

    $S = \{\}$


| Constraint | Constraint w/ substitutions | Updated substitution map | Comment |
| --- | --- | --- | --- |
| $t_g = t_x \rightarrow t_{g(x)}$ | | $\color{blue}{S = \{t_g \mapsto t_x \rightarrow t_{g(x)} \}}$ | |
| $t_{g(x)} = \texttt{num}$ | | $\color{blue}{S = \{t_g \mapsto t_x \rightarrow \texttt{num}, t_{g(x)} \mapsto \texttt{num} \}}$ | |
| $\texttt{num} = \texttt{num}$ | | $\color{blue}{S = \{t_g \mapsto t_x \rightarrow \texttt{num}, t_{g(x)} \mapsto \texttt{num} \}}$ | $\color{blue}{\text{Tautology, continue to next constraint}}$ |
| $t_{y} = \texttt{num}$ | | $\color{blue}{S = \{t_g \mapsto t_x \rightarrow \texttt{num}, t_{g(x)} \mapsto \texttt{num}, t_y \mapsto \texttt{num} \}}$ | |
| $t_f = t_h \rightarrow t_{f(h)}$ | | $\color{blue}{S = \{t_g \mapsto t_x \rightarrow \texttt{num}, t_{g(x)} \mapsto \texttt{num}, t_y \mapsto \texttt{num}, t_f \mapsto t_h \rightarrow t_{f(h)}  \}}$ | |
| $t_{f(h)} = \texttt{num} \rightarrow t_{f(h)(5)}$ | | $\color{blue}{S = \{\\ t_g \mapsto t_x \rightarrow \texttt{num},\\ t_{g(x)} \mapsto \texttt{num},\\ t_y \mapsto \texttt{num},\\ t_f \mapsto t_h \rightarrow \texttt{num} \rightarrow t_{f(h)(5)},\\ t_{f(h)} \mapsto \texttt{num} \rightarrow t_{f(h)(5)} \\ \}}$ | |
| $t_{h} = t_y \rightarrow \texttt{num}$ | $\color{blue}{t_{h} = \texttt{num} \rightarrow \texttt{num}}$ |  | |
| $t_{f} = t_g \rightarrow t_x \rightarrow \texttt{num}$ | $\color{blue}{t_f = \texttt{num} \rightarrow \texttt{num} \rightarrow \texttt{num} \rightarrow \texttt{t_{f(h)(5)}}$  |  |  |

In [0]:
th = num => num
tf = 
num -> num -> num -> tfh5 = tx -> num -> tx -> num

cmd0.sc:1: not found: value th
val res0_0 = th = num => num
             ^cmd0.sc:2: not found: value tf
val res0_1 = tf = 
             ^

: 